In [3]:
from PIL import Image
import glob
import torchvision
import numpy as np
import csv
import os

In [4]:
def get_padding(image):
    w, h = image.shape[0:2]
    max_dim = max([w,h])
    horz = (max_dim - w) / 2
    vert = (max_dim - h) / 2
    l_pad = int(np.ceil(horz))
    r_pad = int(np.floor(horz))
    t_pad = int(np.ceil(vert))
    b_pad = int(np.floor(vert))
    return (t_pad, r_pad, b_pad, l_pad)

In [5]:
def get_transforms():
    for cat in ["other", "martin"]:
        images = glob.glob(f'images/raw/{cat}/*.jpg')
        for index, image in enumerate(images):
            im = Image.open(image)
            imarray = np.array(im)
            im_pad = torchvision.transforms.functional.pad(im, get_padding(imarray))
            im_sized = torchvision.transforms.functional.resize(im_pad, 224)
            im_sized.save(f'images/transformed/{cat}/{index}.jpg')

In [6]:
def create_csv():
    with open('train.csv', 'w') as csvfile:
        writer = csv.writer(csvfile, lineterminator='\n')
        writer.writerow(['image', 'label'])
        for index, cat in enumerate(["other", "martin"]):
            for image in glob.glob(f'images/transformed/{cat}/*.jpg'):
                writer.writerow([index, image])

NameError: name 'main' is not defined